In [26]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome(r'C:\Users\Liad\chromedriver.exe')
address = 'http://play.typeracer.com'
driver.get(address)

# Option 1 - Login And Scrape From Homepage

The sited has a problem loading a lot of resluts so it does not work

In [197]:
def login(username, password):
    driver.find_element_by_class_name('gwt-Anchor').click()
    username_element = driver.find_element_by_name('username')
    password_element = driver.find_element_by_name('password')
    username_element.clear()
    password_element.clear()
    username_element.send_keys(username)
    password_element.send_keys(password)
    driver.find_element_by_class_name('gwt-Button').click()

In [198]:
stats_window = driver.find_element_by_class_name('statsView')
tab_bar = stats_window.find_element_by_class_name('gwt-TabBar')
tab_labels = tab_bar.find_elements_by_class_name('gwt-Label')
for label in tab_labels:
    if label.text == 'My Scores':
        label.click()

In [281]:
def get_window_stats():
    stats_window = driver.find_element_by_class_name('statsView')
    user_stats = stats_window.find_element_by_class_name('UserGameResultsView').find_element_by_class_name('StatsTable').find_element_by_tag_name('table')
    rows = user_stats.find_elements_by_tag_name('tr')
    for row in rows[1:]:
        row.find_element_by_tag_name('a').click()
        if detect_loading_error():
            time.sleep(3)
            row.find_element_by_tag_name('a').click()
        race_info = driver.find_element_by_class_name('popupContent')
        text_info = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.CLASS_NAME, 'textInfoAttribution')))
        results = [x.text for x in race_info.find_elements_by_class_name('tblOwnStatsNumber')]
        full_text = race_info.find_element_by_class_name('fullTextStr').text
        info = text_info.text
        detailed_info = [x.text for x in text_info.find_elements_by_css_selector('*')]
        races.append({'results': results, 'full_text': full_text, 'text_info': info, 'detailed_info': detailed_info})
        driver.find_element_by_class_name('xButton').click()
        yield {'results': results, 'full_text': full_text, 'text_info': info, 'detailed_info': detailed_info}

In [230]:
def next_list():
    # stats_window = driver.find_element_by_class_name('statsView')
    stats_window = driver.find_element_by_class_name('statsView')
    #element_present = EC.presence_of_element_located((By.CLASS_NAME, 'gwt-Anchor'))
    elements = stats_window.find_elements_by_class_name('gwt-Anchor')
    for item in elements:
        if item.text == 'older':
            item.click()

In [217]:
for _ in range(200):
    next_list()

In [206]:
next_list()

In [210]:
import time

In [229]:
def detect_loading_error():
    # Loading error occurs after couple of times of pressing older
    # In order to fix the error the user must finish a race
    try:
        popup = driver.find_element_by_class_name('popupContent')
        message = popup.find_element_by_tag_name('span')
        if message.text == 'Warning':
            popup.find_element_by_class_name('xButton').click()
            return True
        return False
    except:
        return False
detect_loading_error()

True

In [238]:
all_stats = []

In [282]:
while not detect_loading_error():
    for race in get_window_stats():
        all_stats.append(race)
    next_list()
    time.sleep(1)

TimeoutException: Message: 


In [268]:
all_stats

[{'results': ['2019 March 4, Monday 20:18:10 UTC+2',
   '50 wpm',
   '0:57',
   '96%',
   '28'],
  'full_text': 'Situational awareness is key, and agents should not only be familiar with the tactical opportunities offered by their immediate environment, but how those opportunities can be exploited to their advantage with the appropriate equipment.',
  'text_info': '— Deus Ex (Game) by EIDOS  ',
  'detailed_info': ['Deus Ex', '']},
 {'results': ['2019 March 4, Monday 19:49:37 UTC+2',
   '54 wpm',
   '0:40',
   '97%',
   '31'],
  'full_text': "Yep, my ex-wife Tammy cheated on me, then we divorced, then last week I ran into her sister Beth here. Turns out she hates Tammy too, so we started dating. It's like a fairy tale.",
  'text_info': '— Parks and Recreation (null) by Universal Studios Home Entertainment  ',
  'detailed_info': ['Parks and Recreation', '']},
 {'results': ['2019 March 3, Sunday 20:36:44 UTC+2',
   '49 wpm',
   '0:50',
   '95%',
   '31'],
  'full_text': 'One mile to the w

In [291]:
len(all_stats)

281

# Option 2 - Using the data site

In [57]:
def get_simple_data(username):
    # Setting the page to 100 entries at at time
    address = f'https://data.typeracer.com/pit/race_history?user={username}&n=20&startDate='
    driver.get(address)
    driver.find_element_by_name('n').find_elements_by_tag_name('option')[-1].click()
    driver.find_element_by_css_selector("[value='Search']").click()
    
    # Looping over all pages
    while True:
        data = []
        rows = driver.find_element_by_class_name('scoresTable').find_elements_by_tag_name('tr')
        for row in rows[1:]:
            race = [x.text for x in row.find_elements_by_css_selector('*')][1:7]
            if race[-1] == 'today':
                # Providing proper timestamp
                race[-1] = date.today().strftime('%b. %d, %Y')       
            data.append(race)

        next = driver.find_elements_by_xpath("//a[contains(text(), 'load older')]")
        if next:
            next[0].click()
            yield data
        else:
            break
    yield data

In [83]:
from datetime import date
import pandas as pd

df = pd.DataFrame()
for page in get_simple_data('typeracer'):
    df = df.append(page)
    
df.set_axis(['race_number', 'speed', 'accuracy', 'points', 'place', 'date'], axis=1 , inplace=True)
df = df.iloc[::-1].reset_index().drop('index', axis=1)
df

,race_number,speed,accuracy,points,place,date
0,1,49 WPM,N/A,N/A,1/4,"July 27, 2008"
1,2,60 WPM,N/A,N/A,1/4,"July 27, 2008"
2,3,37 WPM,N/A,N/A,3/3,"July 27, 2008"
3,4,65 WPM,N/A,N/A,1/4,"July 28, 2008"
4,5,72 WPM,N/A,N/A,1/3,"July 28, 2008"
...,...,...,...,...,...,...
1219,1225,74 WPM,99.2%,66,3/3,"May 5, 2019"
1220,1226,65 WPM,97.9%,51,4/4,"May 5, 2019"
1221,1227,69 WPM,98.2%,58,3/3,"July 21, 2019"
1222,1228,59 WPM,97.0%,26,2/5,"Aug. 16, 2019"


# Option 3 - Using exact link

In [2]:
def wait_click_element(element, seconds):
    start_time = time.time()
    while time.time() < start_time + seconds:
        try:
            element.click()
            return
        except:
            pass

In [19]:
def get_page_data():
    race_details = driver.find_element_by_class_name('raceDetails')
    result = {}
    for row in race_details.find_elements_by_tag_name('tr'):
        cells = row.find_elements_by_tag_name('td')
        result[cells[0].text] = cells[1].text
    result['Speed'] = ' '.join(result['Speed'].split(' ')[:-2])
    result['Participants'] = result['Rank'][-2]
    result['Rank'] = result['Rank'][0]
    text_info = driver.find_element_by_class_name('fullTextStr').find_element_by_xpath('./..').text
    result['Full_Text'] = text_info.split('\n')[0]
    book, author = text_info.split('\n')[1][2:-12].split(' by ', 1)
    result['Title'] = book[:book.rfind('(') - 1]
    result['Genre'] = book[book.rfind('(') + 1:book.rfind(')')]
    result['Author'] = author

    def find_typing_error():
        expected_char = driver.find_element_by_class_name('remainingChars').text[0]
        typed_char = driver.find_element_by_class_name('errChar').text
        return f'{typed_char} instead of {expected_char}'

    error_li = []
    mistakes = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'WordsWithErrors')))
    for row in mistakes.find_elements_by_tag_name('li'):
        try:
            wait_click_element(row, 10)
            row.click()
            error_li.append(tuple([row.text, find_typing_error()]))
        except:
            error_li = []
            break

    result['errors'] = error_li
    return result

In [27]:
def scrape_user(username, start_number=1, number_of_races=None):
    data = []
    user_link = f'https://data.typeracer.com/pit/result?id=|tr:{username}|'
    if not number_of_races:
        driver.get(f'https://data.typeracer.com/pit/race_history?user={username}&n=20&startDate=')
        number_of_races = int(driver.find_element_by_class_name(
            'scoresTable').find_elements_by_tag_name('tr')[1].text.split(' ')[0]) + 1
        
    for x in range(start_number, number_of_races):
        link = user_link + str(x)
        time.sleep(2)
        driver.get(link)
        data.append(get_page_data())
    
    return data

In [28]:
import pandas as pd
from os import listdir

In [29]:
class Typer():
    
    def __init__(self, username):
        self.username = username
        self.filename = username + '.csv'
        if self.filename in listdir():
            self.df = pd.read_csv(self.filename)
        else:
            self.df = None
            
    def update_user(self, count=None):
        if self.df is None:
            self.df = scrape_user(self.username)
        else:
            new_data = scrape_user(self.username, self.df['Race Number'].apply(int).max() + 1, count)
            self.df = self.df.append(new_data, ignore_index=True)
        self.df.to_csv(self.filename, index=False)

In [33]:
user = Typer('ozonzono')
user.update_user()
user.df

,Accuracy,Author,Date,Full_Text,Genre,Opponents,Participants,Race Number,Racer,Rank,Speed,Title,errors
0,93.80%,Garret Keizer,"Sat, 22 Dec 2018 18:19:52 +0000","It is always important, I think, to be clear a...",book,NaN,5,1,Liad (ozonzono),4,30 WPM,World Enough and Time,"[('think,', ' instead of ,'), ('clear', 'k ins..."
1,93.50%,Wendelin Van Draanen,"Sat, 22 Dec 2018 18:21:21 +0000","A million dollars sounds like a lot, I know. E...",book,dakid02 (3rd place),5,2,Liad (ozonzono),4,28 WPM,Sammy Keyes and the Dead Giveaway,"[('million', 'o instead of i'), ('know.', 'i i..."
2,93.60%,James Joyce,"Sat, 22 Dec 2018 18:22:31 +0000","Yes, it was her he was looking at, and there w...",book,howtokillamokingbird (2nd place),5,3,Liad (ozonzono),3,34 WPM,Ulysses,"[('Yes,', '. instead of ,'), ('was', 'a instea..."
3,93.50%,"John F. Hughes, Andries van Dam, James D. Fole...","Sat, 22 Dec 2018 18:24:32 +0000",The process of determining what pixels should ...,book,NaN,4,4,Liad (ozonzono),3,31 WPM,Computer Graphics: Principles and Practice,"[('process', 's instead of c'), ('determining'..."
4,93.70%,Dan Harris,"Sat, 22 Dec 2018 18:26:15 +0000","Look at the stars. Same stars as last week, la...",movie,NaN,5,5,Liad (ozonzono),4,33 WPM,Imaginary Heroes,"[('Same', 's instead of S'), ('as', ' instead ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8058,96.9%,George Michael,"Thu, 27 Feb 2020 11:44:04 +0000",I can't make you love me if you don't. You can...,song,NaN,5,8060,Liad (ozonzono),1,84 WPM Try,I Can't Make You Love Me,"[(don't., instead of .), (feel, l instead of ..."
8059,96.7%,ABC Studios,"Thu, 27 Feb 2020 11:45:30 +0000",We may never know who killed Paige. But this i...,other,kjax (2nd place) nuke4duke (4th place),5,8061,Liad (ozonzono),1,76 WPM Try,Scandal,"[(this, instead of s), (isn't, instead of s)..."
8060,97.7%,Key,"Thu, 27 Feb 2020 11:46:24 +0000",I hate this town. It's too filled with memorie...,game,flip2theleft (1st place) nesbmido (2nd place) ...,5,8062,Liad (ozonzono),4,79 WPM Try,Clannad,"[(filled, e instead of l), (forget., instead ..."
8061,97.0%,Grandaddy,"Thu, 27 Feb 2020 11:47:27 +0000","Don't change your name, keep it the same for f...",song,cypher_xd (2nd place) honeyageracc (1st place)...,5,8063,Liad (ozonzono),4,80 WPM Try,A.M. 180,"[(keep, instead of p), (fear, h instead of f)..."
